In [ ]:
%load_ext autoreload
%autoreload 2
from IPython import get_ipython; ipython = get_ipython()
import sys, os
from os.path import join as pjoin
import seaborn as sns
p = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'state-space-adaptation' )
sys.path.append(p)
sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))
script_name = pjoin(scripts_dir,'taumodels/test_fit.py')

from config2 import subjects as subjects_NIH
from figure.plots import relplot_multi
from config2 import path_fig
import warnings, gc
from behav_proc import aggRows

In [ ]:
dfbc = []
args_all = []
n_runs = 30
#fit_to = 'errors'
fit_to = 'errors_and_err_sens'
#%debug
#for subj in subjects_NIH[:6]:
for subj in subjects_NIH:    
    #for nb in [5,10,20,30]:
    #for nb in [10,20]:
    #for nb in [20]:
    for nb in [-1]:
        #for reg in [0.,0.1, 0.25, 0.5, 1.]:
        #for reg in [0.25]:
        #for reg in [0, 0.25]:
        for reg in [0, 0.1]:
            for fitmask in ['no','random', 'stable']:
            #for fitmask in ['no']:
                ipython.run_line_magic('run', f'-i {script_name} '
                    f'--subject {subj} --reg {reg:.2f} '
                    f'--save_suffix _reg={reg:.2f}_mask={fitmask}_nb={nb} '
                    f' --fitmask {fitmask} --n_runs {n_runs} '
                    f'--exit_after args_prep --num_bases {nb}'
                    f' --fit_to {fit_to}')
                #dfbc += [bestCalcs]
                args_all += args_ext
    
# for subj in subjects_NIH:
#     ipython.run_line_magic('run', f'-i {script_name} '
#        f'--subject {subj} --reg 1')
#     dfbc += [bestCalcs]

In [ ]:
import gc;gc.collect()

In [ ]:
print(len(args_all))
n_jobs_for_all = 50
if n_jobs_for_all == 1:
    plr = []
    for arg in args_all:
        plr += [_opt(**arg)]
else:
    plr = Parallel(n_jobs=n_jobs_for_all, backend='multiprocessing',
                )(delayed(_opt)(**arg) for arg in args_all) 

In [ ]:
from behav_proc import aggRows
plr = pd.DataFrame(plr, columns=['minres','pard','arg','subject']) 
for k in plr.arg.values[0]:
    plr[k] = plr.arg.apply(lambda x: x.get(k,None))

def f(x):
    thr = x["small_err_thr"]
    
    r = (f'useTrueErr={x["use_true_errors"]}; optimES0={x["optimize_initial_err_sens"]};'
    f' capES={x["cap_err_sens"]}; '
    f' fit_to={x["fit_to"]}; {x["rnsuff"]}'     )
    return r
#sethr={thr > 1e-6}; 
plr['runname'] = plr.apply(f,1)

###################################################
plrg = plr[~plr.pard.isnull()].copy()

for k in plrg.pard.values[0]:
    plrg[k] = plrg.pard.apply(lambda x: x.get(k,None))


plrg['fun'] = plrg.apply(lambda x: x['minres'].fun, 1)                   



bestCalcs = aggRows(plrg, 'fun', 'min',
     grp = plrg.groupby(['subject','runname'] ) ).reset_index()

bestCalcs['fitmask'] = bestCalcs['runname'].str.extract(r'_mask=([^_]*)')

bestCalcs.to_pickle(pjoin(path_data,'herz_param_calced6.pkl.zip')
    ,compression='zip')



In [ ]:
sz = plrg.groupby(['subject','runname']).size()
mx = sz.max() 
assert mx <= argscmd.n_runs, mx

## best

In [ ]:
#bestCalcs.sort_values(['runname'], ascending=False)['runname']

In [ ]:
rn = sz[sz > argscmd.n_runs].index[0]
cols = [col for col in plrg.columns if col not in ['minres','pard','arg']]
plrg.query('runname == @rn')[cols]

In [ ]:
bc = bestCalcs.groupby(['runname'])['fun'].mean().to_frame()
i = bc['fun'].idxmin()
print( bc.loc[i] )

cols = ['eta','alpha','alpha_w','weight_forgetting_exp_pause','initial_sensitivity','fun']
bc00 = bestCalcs.query('use_true_errors == True and fitmask != "random" and reg > 0')
bc0 = bc00.groupby(['runname'])
bc = bc0['fun'].mean().to_frame()
i = bc['fun'].idxmin()
print( bc.loc[i] )
rn = bc.loc[i].name

In [ ]:
plr.query('subject.isin(@bad_subjects) and runname == @rn').iloc[0].minres

In [ ]:
bad_subjects = set(subjects_NIH) - set( bc00.query('runname == @rn')['subject'].values )
bad_subjects = list(bad_subjects)
print(bad_subjects)

In [ ]:
#rn = 'useTrueErr=True; optimES0=optimize; capES=False; fit_to=errors; _reg=0.25_mask=stable_nb=-1_sethr=mestd*1.0'

In [ ]:
bc0[cols].mean().nsmallest(10,columns=['fun'])[cols]

In [ ]:
dfd = bestCalcs.query('use_true_errors == True and runname == @rn').\
    groupby(['runname'])[cols].describe(percentiles=[])

In [ ]:
dfd.T

In [ ]:
dfd['eta']['mean']

In [ ]:
#bestCalcs['fitmask']

In [ ]:
bestCalcs.columns

In [ ]:
# df_ = dfcc_all_NIH_.query('trial_shift_size == 1 and '
#     'trial_group_col_calc == "trials" and retention_factor_s == "0.924"'
#                          )
# df_.loc[df_['pre_break_duration'] > 0,
#         ['subject','trials','pre_break_duration']]

# plot

In [ ]:
assert not bestCalcs.duplicated(['subject','runname']).any()

In [ ]:
dfcc_all_NIH_['perturbation_pscadj'] = dfcc_all_NIH_['perturbation']
dfcc_all_NIH_.loc[dfcc_all_NIH_['pert_seq_code'] == 1, 
    'perturbation_pscadj'] = -dfcc_all_NIH_['perturbation']

#df0['perturbation'].describe()

dfr =[]
for rowi,row in bestCalcs.iterrows():
    runname = row['runname']
    pard_ = row['pard']
    if pard_ is None:
        print('empty',runname)
        continue
    pard = pard_.copy()
    if 'fun' in pard:
        del pard['fun']
        del pard['nit']
        del pard['nfev']        
    
    subj = row["subject"]
    print(rowi,len(bestCalcs), subj, runname)
    df0 = dfcc_all_NIH_.query('trial_shift_size == 1 and '
        'trial_group_col_calc == "trials" and '
        'retention_factor_s == "0.924" and '
        'subject == @subj').sort_values(['trials'])
    pre_break_duration = df0['pre_break_duration'].values
    error = df0['error_pscadj'].values * 180 / np.pi
    perturb = df0['perturbation_pscadj'].values

    use_true_errors = row['use_true_errors']
    cap_err_sens = row['cap_err_sens']    
    r = simMoE(-perturb,EC_mask=EC_mask,true_errors=error,
                pre_break_duration=pre_break_duration,
                **pard, cap_err_sens=cap_err_sens,
               small_err_thr = row['small_err_thr'],
               num_bases = row['num_bases'],
        use_true_errors=use_true_errors)
    
    (motor_output,error_pred2,ws,err_sens2, 
        gaussian_centers,gaussian_variances) = r 
    _dat = np.array((motor_output,error_pred2,
        err_sens2, np.arange(len(motor_output), dtype=int) ) )
    df_ = pd.DataFrame(data=_dat.T, 
        columns = ['motor_output','error_pred2',
                'err_sens2','trial_index'])
            
    df_['err_sens'] = df0['err_sens'].values
    df_['error_pscadj_deg'] = df0['error_pscadj'].values * 180 / np.pi

    for kn,kv in row.items():
        df_[kn] = kv
    dfr += [df_ ]
dfr = pd.concat(dfr,ignore_index=1)

dfr = dfr.sort_values(['runname'], ascending=False)

runnames = dfr['runname'].unique()
print(runnames)

gc.collect()

In [ ]:
print( plr.subject.nunique(), dfr.subject.nunique(), len(runnames) )

In [ ]:
gc.collect()

In [ ]:
#%debug
import warnings

for rn in list(sorted(runnames))[::-1]:
    df_ = dfr.query('runname == @rn')# and fit_to == "errors"')
    
    alphas = bestCalcs.query(f'runname == @rn')['alpha']
    ss = f'\n  {alphas.mean():.2f} {alphas.std():.2f}' 
    
    if len(df_) == 0:
        print('Emtpy for ',rn)
        continue
    with warnings.catch_warnings(record=True):
        warnings.filterwarnings("ignore",category=FutureWarning)
        warnings.filterwarnings("ignore",category=UserWarning)
        fg,df__ = relplot_multi(kind='line',data=df_,
                x='trial_index',
            ys=[['error_pred2','error_pscadj_deg'],
               ['err_sens','err_sens2']], 
            ylabel = ['error','ES'],
            ylim=[(-100,100), (-0.8,1)], aspect = 2, 
                                height = 3,
            facet_kws = {'sharex':True, 'sharey':False})

        fg.refline(y=0)    
        fg.refline(x=192)
        fg.refline(x=192*2)
        fg.refline(x=192*3)
        plt.suptitle(rn + '\n' + ss)    

        plt.savefig(pjoin(path_fig, rn + '.pdf'))
        plt.close()